In [2]:
# import pylab as plt
import tqdm as tqdm
import pandas as pd
from herbie import FastHerbie
from herbie import Herbie
# import cartopy.crs as ccrs
import numpy as np
import xarray as xr
import warnings
warnings.filterwarnings('ignore')
import zarr
from IPython.utils import io

In [3]:
#Specify the time range over which to generate zarr files
#DATES = pd.date_range('2021-03-10 15:00', '2021-03-10 20:00', freq='1H')
DATES = pd.date_range('2021-03-15 00:00', '2021-03-17 00:00', freq='1H')

In [4]:
for d in DATES:
    print(d)

2021-03-15 00:00:00
2021-03-15 01:00:00
2021-03-15 02:00:00
2021-03-15 03:00:00
2021-03-15 04:00:00
2021-03-15 05:00:00
2021-03-15 06:00:00
2021-03-15 07:00:00
2021-03-15 08:00:00
2021-03-15 09:00:00
2021-03-15 10:00:00
2021-03-15 11:00:00
2021-03-15 12:00:00
2021-03-15 13:00:00
2021-03-15 14:00:00
2021-03-15 15:00:00
2021-03-15 16:00:00
2021-03-15 17:00:00
2021-03-15 18:00:00
2021-03-15 19:00:00
2021-03-15 20:00:00
2021-03-15 21:00:00
2021-03-15 22:00:00
2021-03-15 23:00:00
2021-03-16 00:00:00
2021-03-16 01:00:00
2021-03-16 02:00:00
2021-03-16 03:00:00
2021-03-16 04:00:00
2021-03-16 05:00:00
2021-03-16 06:00:00
2021-03-16 07:00:00
2021-03-16 08:00:00
2021-03-16 09:00:00
2021-03-16 10:00:00
2021-03-16 11:00:00
2021-03-16 12:00:00
2021-03-16 13:00:00
2021-03-16 14:00:00
2021-03-16 15:00:00
2021-03-16 16:00:00
2021-03-16 17:00:00
2021-03-16 18:00:00
2021-03-16 19:00:00
2021-03-16 20:00:00
2021-03-16 21:00:00
2021-03-16 22:00:00
2021-03-16 23:00:00
2021-03-17 00:00:00


In [5]:
# Create zarr filename prefix
of_name = DATES[0].strftime("%y-%m-%d-%Hz") + '_to_' + DATES[-1].strftime("%y-%m-%d-%Hz")

In [6]:
# Instantiate herbie classes
H = FastHerbie(DATES, model="hrrr", product="sfc", save_dir='/home/xiao169/scratch/hrrr_data', fxx=[1])
# Hnat = FastHerbie(DATES, model="hrrr", product="nat", save_dir='/home/xiao169/scratch/', fxx=[1], verbose=True)

In [7]:
# Define surface variables to be read and download them if needed
surface_name = {} 
surface_name['VIL'] = "VIL"
surface_name['SST'] = ":TMP:surface"  
surface_name['T2m'] ="TMP:2 m"
surface_name['QV2m'] ="SPFH:2 m"
surface_name['U80'] = "UGRD:80 m"
surface_name['V80'] = "VGRD:80 m"
surface_name['U10'] = "UGRD:10 m"
surface_name['V10'] = "VGRD:10 m"
surface_name['PSFC'] = "PRES:surface"
surface_name['LHTFL'] = "LHTFL:surface"
surface_name['SHTFL'] = "SHTFL:surface"
surface_name['DSWRF'] = "DSWRF:surface"
surface_name['LCDC'] = "LCDC"
surface_name['WIND'] = "WIND"
surface_name['HPBL'] = "HPBL"
surface_name['LAND'] = "LAND"
surface_name['DLWRF'] = "DLWRF:surface"
for name, regex in tqdm.tqdm(surface_name.items()):
    H.download(regex)

  0%|          | 0/17 [00:00<?, ?it/s]

👨🏻‍🏭 Created directory: [/home/xiao169/scratch/hrrr_data/hrrr/20210315]


100%|██████████| 17/17 [00:38<00:00,  2.24s/it]


In [8]:
# Convert downloaded grib2 files to zarr
for name, regex in tqdm.tqdm(surface_name.items()):
    with io.capture_output() as captured:
        ds = H.xarray(regex, max_threads=1)
    ds.to_zarr('/home/xiao169/scratch/'+of_name + '_surface.zarr', mode='a')
    ds.close()

100%|██████████| 17/17 [11:49<00:00, 41.71s/it]


In [4]:
# Define native model level fields to be downloaded
level_name = {} 
level_name["U"] = ":(?:U)GRD:[0-9]+ hybrid"
level_name["V"] = ":(?:V)GRD:[0-9]+ hybrid"
level_name["P"] = ":(?:PRES):[0-9]+ hybrid"
level_name["T"] = ":(?:TMP):[0-9]+ hybrid"
level_name["QV"] = ":(?:SPFH):[0-9]+ hybrid"
level_name["QC"] = ":(?:CLMR):[0-9]+ hybrid"
level_name["QI"] = ":(?:CIMIXR):[0-9]+ hybrid"
level_name["Z"] =":(?:HGT:)[0-9]+ hybrid"

# for name, regex in tqdm.tqdm(level_name.items()):
#     Hnat.download(regex)

In [7]:
list(level_name.values())

[':(?:U)GRD:[0-9]+ hybrid',
 ':(?:V)GRD:[0-9]+ hybrid',
 ':(?:PRES):[0-9]+ hybrid',
 ':(?:TMP):[0-9]+ hybrid',
 ':(?:SPFH):[0-9]+ hybrid',
 ':(?:CLMR):[0-9]+ hybrid',
 ':(?:CIMIXR):[0-9]+ hybrid',
 ':(?:HGT:)[0-9]+ hybrid']

In [49]:
# for DATE in tqdm.tqdm(DATES):
    # with io.capture_output() as captured:
new_data = True
iseg = np.arange(0, len(DATES) ,2)
if np.mod(len(DATES), 2) == 1:
    iseg = np.delete(iseg, -1, None)
for i in iseg:
    if i == iseg[-1]:
        seg = DATES[i:]
    else:
        seg = DATES[i:i+2]
    for s in seg:
        print(s)
    Hnat = FastHerbie(seg, model="hrrr", product="nat", save_dir='/home/xiao169/scratch/', fxx=[1], verbose=True)
    searchstring = f"({'|'.join(list(level_name.values()))})"
    m = Hnat.xarray(searchstring, max_threads=4)
    # m.drop_vars('gribfile_projection')
    # del m.attrs['local_grib'] 
    # del m.attrs['remote_grib'] 
    # print(m)
    if new_data: 
        m.to_zarr('/home/xiao169/scratch/' + of_name + '_levels.zarr', mode="a")
        new_data = False
    else:
        m.to_zarr('/home/xiao169/scratch/' + of_name + '_levels.zarr',append_dim="time")
    m.close() 

2021-03-08 00:00:00
2021-03-08 01:00:00
✅ Found ┊ model=hrrr ┊ product=nat ┊ 2021-Mar-08 01:00 UTC F01 ┊ GRIB2 @ local ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=nat ┊ 2021-Mar-08 00:00 UTC F01 ┊ GRIB2 @ local ┊ IDX @ aws
2021-03-08 02:00:00
2021-03-08 03:00:00
2021-03-08 04:00:00
✅ Found ┊ model=hrrr ┊ product=nat ┊ 2021-Mar-08 03:00 UTC F01 ┊ GRIB2 @ local ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=nat ┊ 2021-Mar-08 04:00 UTC F01 ┊ GRIB2 @ local ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=nat ┊ 2021-Mar-08 02:00 UTC F01 ┊ GRIB2 @ local ┊ IDX @ aws


In [51]:
ds = xr.open_zarr('~/scratch/21-03-08-00z_to_21-03-16-00z_surface.zarr')

In [56]:
ds_nat = xr.open_zarr('~/scratch/21-03-08-00z_to_21-03-09-00z_levels.zarr')

In [60]:
ds.valid_time.values

array(['2021-03-08T01:00:00.000000000', '2021-03-08T02:00:00.000000000',
       '2021-03-08T03:00:00.000000000', '2021-03-08T04:00:00.000000000',
       '2021-03-08T05:00:00.000000000', '2021-03-08T06:00:00.000000000',
       '2021-03-08T07:00:00.000000000', '2021-03-08T08:00:00.000000000',
       '2021-03-08T09:00:00.000000000', '2021-03-08T10:00:00.000000000',
       '2021-03-08T11:00:00.000000000', '2021-03-08T12:00:00.000000000',
       '2021-03-08T13:00:00.000000000', '2021-03-08T14:00:00.000000000',
       '2021-03-08T15:00:00.000000000', '2021-03-08T16:00:00.000000000',
       '2021-03-08T17:00:00.000000000', '2021-03-08T18:00:00.000000000',
       '2021-03-08T19:00:00.000000000', '2021-03-08T20:00:00.000000000',
       '2021-03-08T21:00:00.000000000', '2021-03-08T22:00:00.000000000',
       '2021-03-08T23:00:00.000000000', '2021-03-09T00:00:00.000000000',
       '2021-03-09T01:00:00.000000000', '2021-03-09T02:00:00.000000000',
       '2021-03-09T03:00:00.000000000', '2021-03-09